In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pathlib
import pickle

sys.path.append('..')

In [3]:
import pandas as pd

In [4]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.PLAGE import PLAGE
from pals.ORA import ORA
from pals.GSEA import GSEA
from pals.common import *

2021-01-07 16:46:33.636 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Beer Analysis

### Load data

Generate token by logging in to PiMP

In [5]:
# username = 'joewandy' # PiMP username
# password = 'enter' # PiMP password
# host = 'localhost:8000' # server address and port
# token = get_authentication_token(host, username, password)

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [6]:
token = get_pimp_API_token_from_env()

In [7]:
analysis_id = 1321 # example beer analysis

In [8]:
int_df, annotation_df, experimental_design = download_from_pimp(token, PIMP_HOST, analysis_id, 'kegg')

2021-01-07 16:46:34.567 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: C:\Users\joewa\AppData\Local\Temp\pimp_analysis_1321.p


In [9]:
int_df

,Beer_1_full1.mzXML,Beer_1_full2.mzXML,Beer_1_full3.mzXML,Beer_2_full1.mzXML,Beer_2_full2.mzXML,Beer_2_full3.mzXML,Beer_3_full1.mzXML,Beer_3_full2.mzXML,Beer_3_full3.mzXML,Beer_4_full1.mzXML,Beer_4_full2.mzXML,Beer_4_full3.mzXML
row_id,,,,,,,,,,,,
3033929,2.235291e+09,2.000478e+09,2.170697e+09,2.242760e+09,2.279882e+09,1.959480e+09,2.079356e+09,2.110473e+09,2.243653e+09,1.817065e+09,1.746443e+09,1.779827e+09
3033930,4.433491e+07,4.287387e+07,4.894853e+07,4.760448e+07,4.217280e+07,3.908452e+07,3.825778e+07,3.770192e+07,4.087189e+07,3.330477e+07,3.153630e+07,3.102410e+07
3033931,1.723985e+09,1.764235e+09,1.585143e+09,1.543961e+09,1.579320e+09,1.555666e+09,1.698130e+09,1.481824e+09,1.508645e+09,1.642510e+09,1.723919e+09,1.697806e+09
3033932,6.254237e+08,6.503417e+08,5.914975e+08,4.635929e+08,4.298382e+08,4.038747e+08,4.292837e+08,3.708761e+08,4.778932e+08,3.903165e+08,4.080995e+08,4.309892e+08
3033933,1.075022e+09,9.293474e+08,1.092635e+09,1.130720e+09,1.118146e+09,1.192834e+09,1.231442e+09,1.262046e+09,1.460653e+09,1.009838e+09,9.085111e+08,9.967176e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
3041299,1.431211e+04,6.565678e+03,1.478325e+04,1.620252e+04,1.748920e+04,1.284756e+04,3.306687e+04,2.476216e+04,2.869417e+04,2.231166e+04,2.164017e+04,2.727751e+04
3041300,2.273721e+04,2.905976e+04,2.756565e+04,3.080164e+04,2.427240e+04,2.517871e+04,2.718543e+04,2.905361e+04,3.170420e+04,2.597168e+04,3.066904e+04,2.884984e+04
3041301,1.760107e+04,2.674373e+04,2.165889e+04,2.121242e+04,1.737357e+04,2.039137e+04,2.296079e+04,2.001743e+04,2.664124e+04,1.791359e+04,1.642459e+04,2.349759e+04


In [10]:
annotation_df

,entity_id
row_id,
3033929,C00148
3036581,C00148
3036855,C00148
3038249,C00148
3033929,C00163
...,...
3040926,C20522
3040929,C20582
3041077,C20499


In [11]:
experimental_design

{'comparisons': [{'case': 'beer1', 'control': 'beer2', 'name': 'beer1/beer2'},
  {'case': 'beer3', 'control': 'beer4', 'name': 'beer3/beer4'}],
 'groups': {'beer4': ['Beer_4_full3.mzXML',
   'Beer_4_full2.mzXML',
   'Beer_4_full1.mzXML'],
  'beer3': ['Beer_3_full3.mzXML', 'Beer_3_full2.mzXML', 'Beer_3_full1.mzXML'],
  'beer2': ['Beer_2_full3.mzXML', 'Beer_2_full1.mzXML', 'Beer_2_full2.mzXML'],
  'beer1': ['Beer_1_full2.mzXML', 'Beer_1_full1.mzXML', 'Beer_1_full3.mzXML']}}

### PALS analysis using KEGG database exported from PiMP

In [12]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG)

2021-01-07 16:46:34.962 | DEBUG    | pals.feature_extraction:__init__:43 - Using PiMP_KEGG as database
2021-01-07 16:46:34.963 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2021-01-07 16:46:34.984 | DEBUG    | pals.feature_extraction:__init__:56 - Mapping pathway to unique ids
2021-01-07 16:46:34.989 | DEBUG    | pals.feature_extraction:__init__:70 - Creating dataset to pathway mapping
2021-01-07 16:46:35.758 | DEBUG    | pals.feature_extraction:__init__:98 - Computing unique id counts


In [13]:
plage = PLAGE(ds, seed=1)
pathway_df = plage.get_results()

2021-01-07 16:46:35.842 | DEBUG    | pals.PLAGE:__init__:29 - PLAGE initialised
2021-01-07 16:46:35.996 | DEBUG    | pals.preprocessing:process:20 - Performing min-value imputation
2021-01-07 16:46:36.008 | DEBUG    | pals.preprocessing:process:36 - Performing row average imputation
2021-01-07 16:46:36.018 | DEBUG    | pals.preprocessing:process:46 - Applying log normalisation
2021-01-07 16:46:36.019 | DEBUG    | pals.preprocessing:process:53 - Scaling the data across the sample: zero mean and unit variance
2021-01-07 16:46:36.085 | DEBUG    | pals.PLAGE:get_plage_activity_df:84 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2021-01-07 16:46:36.085 | DEBUG    | pals.PLAGE:get_plage_activity_df:85 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2021-01-07 16:46:36.265 | DEBUG    | pals.PLAGE:set_up_resample_plage_p_df:96 - Calculating plage p-values with resampling
2021-01-07 16:46:36.266 | DEBUG    | pals.PLAGE:set_up_resample_plage_p_df:103 - Comparis

In [14]:
pathway_df.sort_values('PiMP_KEGG beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,PiMP_KEGG beer1/beer2 comb_p,PiMP_KEGG beer3/beer4 comb_p
map04740,Olfactory transduction,0.023642,0.407746,5,1,20.00,7.168639e-01,1.11,22.20,0.023642,0.407746
map07016,Sulfonamide derivatives - sulfa drugs,0.045014,1.000000,2,1,50.00,3.961500e-01,0.45,22.50,0.045014,1.000000
map05146,Amoebiasis,0.067249,0.054305,8,2,25.00,5.620670e-01,1.78,22.25,0.067249,0.054305
map00380,Tryptophan metabolism,0.073098,0.059317,64,34,53.12,5.093550e-08,14.26,22.28,0.073098,0.059317
map00460,Cyanoamino acid metabolism,0.080293,0.052164,40,25,62.50,4.257882e-08,8.92,22.30,0.080293,0.052164
...,...,...,...,...,...,...,...,...,...,...,...
map05032,Morphine addiction,1.000000,0.083903,8,3,37.50,2.552558e-01,1.78,22.25,1.000000,0.083903
map05211,Renal cell carcinoma,1.000000,0.066596,3,2,66.67,1.267855e-01,0.67,22.33,1.000000,0.066596
map00254,Aflatoxin biosynthesis,1.000000,1.000000,24,3,12.50,9.288490e-01,5.35,22.29,1.000000,1.000000
map01040,Biosynthesis of unsaturated fatty acids,1.000000,0.082694,42,4,9.52,9.914451e-01,9.36,22.29,1.000000,0.082694


### PALS analysis using KEGG database exported from Reactome

In [15]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_REACTOME_KEGG, 
                reactome_species=REACTOME_SPECIES_HOMO_SAPIENS, reactome_metabolic_pathway_only=True)

2021-01-07 16:46:39.080 | DEBUG    | pals.feature_extraction:__init__:43 - Using COMPOUND as database
2021-01-07 16:46:39.081 | DEBUG    | pals.loader:load_data:84 - Loading ..\pals\data\reactome\metabolic_pathways\COMPOUND\Homo sapiens.json.zip
2021-01-07 16:46:39.100 | DEBUG    | pals.feature_extraction:__init__:56 - Mapping pathway to unique ids
2021-01-07 16:46:39.101 | DEBUG    | pals.feature_extraction:__init__:70 - Creating dataset to pathway mapping
2021-01-07 16:46:39.952 | DEBUG    | pals.feature_extraction:__init__:98 - Computing unique id counts


In [16]:
plage = PLAGE(ds, seed=1)
pathway_df = plage.get_results()

2021-01-07 16:46:40.046 | DEBUG    | pals.PLAGE:__init__:29 - PLAGE initialised
2021-01-07 16:46:40.122 | DEBUG    | pals.preprocessing:process:20 - Performing min-value imputation
2021-01-07 16:46:40.130 | DEBUG    | pals.preprocessing:process:36 - Performing row average imputation
2021-01-07 16:46:40.140 | DEBUG    | pals.preprocessing:process:46 - Applying log normalisation
2021-01-07 16:46:40.142 | DEBUG    | pals.preprocessing:process:53 - Scaling the data across the sample: zero mean and unit variance
2021-01-07 16:46:40.214 | DEBUG    | pals.PLAGE:get_plage_activity_df:84 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2021-01-07 16:46:40.214 | DEBUG    | pals.PLAGE:get_plage_activity_df:85 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2021-01-07 16:46:40.265 | DEBUG    | pals.PLAGE:set_up_resample_plage_p_df:96 - Calculating plage p-values with resampling
2021-01-07 16:46:40.266 | DEBUG    | pals.PLAGE:set_up_resample_plage_p_df:103 - Comparis

In [17]:
pathway_df.sort_values('COMPOUND beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,COMPOUND beer1/beer2 comb_p,COMPOUND beer3/beer4 comb_p
R-HSA-2142770,Synthesis of 15-eicosatetraenoic acid derivatives,0.011792,0.000964,11,1,9.09,0.990381,3.74,34.00,0.011792,0.000964
R-HSA-9020265,Biosynthesis of aspirin-triggered D-series res...,0.011792,0.000964,7,1,14.29,0.946866,2.38,34.00,0.011792,0.000964
R-HSA-2408499,Formation of selenosugars for excretion,0.011792,0.000964,4,1,25.00,0.811478,1.36,34.00,0.011792,0.000964
R-HSA-2142712,Synthesis of 12-eicosatetraenoic acid derivatives,0.011792,0.000964,6,1,16.67,0.918842,2.04,34.00,0.011792,0.000964
R-HSA-9023661,Biosynthesis of E-series 18(R)-resolvins,0.011792,0.000964,7,1,14.29,0.946866,2.38,34.00,0.011792,0.000964
...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-6806664,Metabolism of vitamin K,1.000000,0.049009,2,1,50.00,0.564581,0.68,34.00,1.000000,0.049009
R-HSA-2046106,alpha-linolenic acid (ALA) metabolism,1.000000,0.049009,14,1,7.14,0.997372,4.76,34.00,1.000000,0.049009
R-HSA-2142700,Synthesis of Lipoxins (LX),1.000000,0.317718,11,1,9.09,0.990381,3.74,34.00,1.000000,0.317718
R-HSA-75105,Fatty acyl-CoA biosynthesis,1.000000,0.054674,18,4,22.22,0.913291,6.11,33.94,1.000000,0.054674


### PALS analysis of compounds by connecting to Reactome

In [18]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_REACTOME_KEGG, 
                reactome_species=REACTOME_SPECIES_HOMO_SAPIENS, reactome_metabolic_pathway_only=True, reactome_query=True)

2021-01-07 16:46:42.729 | DEBUG    | pals.feature_extraction:__init__:43 - Using COMPOUND as database
2021-01-07 16:46:42.730 | DEBUG    | pals.loader:load_data:56 - Retrieving data for Homo sapiens from Reactome COMPOUND metabolic_pathway_only=True
2021-01-07 16:46:46.866 | DEBUG    | pals.feature_extraction:__init__:56 - Mapping pathway to unique ids
2021-01-07 16:46:46.867 | DEBUG    | pals.feature_extraction:__init__:70 - Creating dataset to pathway mapping
2021-01-07 16:46:47.705 | DEBUG    | pals.feature_extraction:__init__:98 - Computing unique id counts


In [19]:
plage = PLAGE(ds, seed=1)
pathway_df = plage.get_results()

2021-01-07 16:46:47.806 | DEBUG    | pals.PLAGE:__init__:29 - PLAGE initialised
2021-01-07 16:46:47.885 | DEBUG    | pals.preprocessing:process:20 - Performing min-value imputation
2021-01-07 16:46:47.893 | DEBUG    | pals.preprocessing:process:36 - Performing row average imputation
2021-01-07 16:46:47.904 | DEBUG    | pals.preprocessing:process:46 - Applying log normalisation
2021-01-07 16:46:47.905 | DEBUG    | pals.preprocessing:process:53 - Scaling the data across the sample: zero mean and unit variance
2021-01-07 16:46:47.970 | DEBUG    | pals.PLAGE:get_plage_activity_df:84 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2021-01-07 16:46:47.971 | DEBUG    | pals.PLAGE:get_plage_activity_df:85 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2021-01-07 16:46:48.022 | DEBUG    | pals.PLAGE:set_up_resample_plage_p_df:96 - Calculating plage p-values with resampling
2021-01-07 16:46:48.022 | DEBUG    | pals.PLAGE:set_up_resample_plage_p_df:103 - Comparis

In [20]:
pathway_df.sort_values('COMPOUND beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,COMPOUND beer1/beer2 comb_p,COMPOUND beer3/beer4 comb_p
R-HSA-2142770,Synthesis of 15-eicosatetraenoic acid derivatives,0.011792,0.000964,11,1,9.09,0.990381,3.74,34.00,0.011792,0.000964
R-HSA-9020265,Biosynthesis of aspirin-triggered D-series res...,0.011792,0.000964,7,1,14.29,0.946866,2.38,34.00,0.011792,0.000964
R-HSA-2408499,Formation of selenosugars for excretion,0.011792,0.000964,4,1,25.00,0.811478,1.36,34.00,0.011792,0.000964
R-HSA-2142712,Synthesis of 12-eicosatetraenoic acid derivatives,0.011792,0.000964,6,1,16.67,0.918842,2.04,34.00,0.011792,0.000964
R-HSA-9023661,Biosynthesis of E-series 18(R)-resolvins,0.011792,0.000964,7,1,14.29,0.946866,2.38,34.00,0.011792,0.000964
...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-6806664,Metabolism of vitamin K,1.000000,0.049009,2,1,50.00,0.564581,0.68,34.00,1.000000,0.049009
R-HSA-2046106,alpha-linolenic acid (ALA) metabolism,1.000000,0.049009,14,1,7.14,0.997372,4.76,34.00,1.000000,0.049009
R-HSA-2142700,Synthesis of Lipoxins (LX),1.000000,0.317718,11,1,9.09,0.990381,3.74,34.00,1.000000,0.317718
R-HSA-75105,Fatty acyl-CoA biosynthesis,1.000000,0.054674,18,4,22.22,0.913291,6.11,33.94,1.000000,0.054674


### ORA Analysis

In [21]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG)

2021-01-07 16:46:50.510 | DEBUG    | pals.feature_extraction:__init__:43 - Using PiMP_KEGG as database
2021-01-07 16:46:50.511 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2021-01-07 16:46:50.535 | DEBUG    | pals.feature_extraction:__init__:56 - Mapping pathway to unique ids
2021-01-07 16:46:50.541 | DEBUG    | pals.feature_extraction:__init__:70 - Creating dataset to pathway mapping
2021-01-07 16:46:51.352 | DEBUG    | pals.feature_extraction:__init__:98 - Computing unique id counts


In [22]:
ora = ORA(ds)
pathway_df = ora.get_results()

2021-01-07 16:46:51.450 | DEBUG    | pals.ORA:__init__:22 - ORA initialised
2021-01-07 16:46:51.619 | DEBUG    | pals.ORA:get_results:47 - Calculating ORA
2021-01-07 16:46:51.620 | DEBUG    | pals.preprocessing:process:20 - Performing min-value imputation
2021-01-07 16:46:51.628 | DEBUG    | pals.preprocessing:process:36 - Performing row average imputation
2021-01-07 16:46:51.640 | DEBUG    | pals.preprocessing:process:46 - Applying log normalisation
2021-01-07 16:46:57.029 | DEBUG    | pals.ORA:get_results:104 - Correcting for multiple t-tests
2021-01-07 16:46:57.033 | DEBUG    | pals.feature_extraction:_calculate_coverage_df:329 - Calculating dataset formula coverage


In [23]:
pathway_df.sort_values('PiMP_KEGG beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,PiMP_KEGG beer1/beer2 comb_p,PiMP_KEGG beer3/beer4 comb_p,unq_pw_F,tot_ds_F,F_coverage
map00350,Tyrosine metabolism,3.621191e-16,1.681454e-16,8.183892e-14,3.800087e-14,53,39,73.58
map00330,Arginine and proline metabolism,1.775486e-12,1.686603e-12,2.006299e-10,1.905861e-10,79,50,63.29
map00290,"Valine, leucine and isoleucine biosynthesis",9.591121e-12,2.028069e-11,7.225312e-10,1.527812e-09,17,16,94.12
map00400,"Phenylalanine, tyrosine and tryptophan biosynt...",2.824918e-10,7.042130e-10,1.596078e-08,3.978804e-08,30,22,73.33
map00660,C5-Branched dibasic acid metabolism,5.693949e-09,1.159169e-08,1.838332e-07,2.619722e-07,21,16,76.19
...,...,...,...,...,...,...,...,...
map00906,Carotenoid biosynthesis,9.596425e-01,9.388042e-01,1.000000e+00,1.000000e+00,66,8,12.12
map00860,Porphyrin and chlorophyll metabolism,9.998171e-01,9.999102e-01,1.000000e+00,1.000000e+00,89,7,7.87
map00901,Indole alkaloid biosynthesis,9.184987e-01,9.340620e-01,1.000000e+00,1.000000e+00,30,4,13.33
map00351,DDT degradation,1.000000e+00,9.831430e-01,1.000000e+00,1.000000e+00,20,2,10.00


### GSEA Analysis

In [24]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG)

2021-01-07 16:46:57.218 | DEBUG    | pals.feature_extraction:__init__:43 - Using PiMP_KEGG as database
2021-01-07 16:46:57.219 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2021-01-07 16:46:57.240 | DEBUG    | pals.feature_extraction:__init__:56 - Mapping pathway to unique ids
2021-01-07 16:46:57.246 | DEBUG    | pals.feature_extraction:__init__:70 - Creating dataset to pathway mapping
2021-01-07 16:46:58.040 | DEBUG    | pals.feature_extraction:__init__:98 - Computing unique id counts


In [25]:
gsea = GSEA(ds)
pathway_df = gsea.get_results()

2021-01-07 16:46:58.144 | DEBUG    | pals.GSEA:__init__:38 - GSEA initialised with num_resamples=1000 and ranking_method=signal_to_noise
2021-01-07 16:46:58.242 | DEBUG    | pals.GSEA:get_results:63 - Calculating GSEA
2021-01-07 16:46:58.243 | DEBUG    | pals.preprocessing:process:20 - Performing min-value imputation
2021-01-07 16:46:58.251 | DEBUG    | pals.preprocessing:process:36 - Performing row average imputation
2021-01-07 16:46:58.284 | DEBUG    | pals.GSEA:get_results:92 - Running comparison case=beer1 control=beer2
2021-01-07 16:47:07.749 | DEBUG    | pals.GSEA:get_results:92 - Running comparison case=beer3 control=beer4
2021-01-07 16:47:16.593 | DEBUG    | pals.feature_extraction:_calculate_coverage_df:329 - Calculating dataset formula coverage


In [26]:
pathway_df.sort_values('PiMP_KEGG beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,PiMP_KEGG beer1/beer2 comb_p,beer1/beer2 ES_score,beer3/beer4 p-value,PiMP_KEGG beer3/beer4 comb_p,beer3/beer4 ES_score,unq_pw_F,tot_ds_F,F_coverage
map00680,Methane metabolism,0.058824,0.063894,-0.279640,0.137566,0.883603,-0.305596,72,20,27.78
map00402,Benzoxazinoid biosynthesis,0.000000,0.075415,-0.839216,0.009901,0.225945,0.717206,9,5,55.56
map00564,Glycerophospholipid metabolism,0.029851,0.083271,-0.392633,0.016129,0.099915,0.326296,23,8,34.78
map05200,Pathways in cancer,0.008696,0.088038,0.729971,0.015267,0.184571,-0.652592,15,4,26.67
map01055,Biosynthesis of vancomycin group antibiotics,0.000000,0.094269,-0.382911,0.367647,0.219774,0.286898,17,6,35.29
...,...,...,...,...,...,...,...,...,...,...
map04140,Regulation of autophagy,0.920290,1.000000,0.409278,0.019608,0.348001,0.836363,2,1,50.00
map00780,Biotin metabolism,0.921053,1.000000,0.202988,0.993289,0.999797,-0.121033,15,6,40.00
map00960,"Tropane, piperidine and pyridine alkaloid bios...",0.950311,1.000000,0.140619,0.550725,0.364679,0.200523,51,25,49.02
map00920,Sulfur metabolism,0.939850,1.000000,0.223856,0.706897,0.795760,-0.312512,27,7,25.93
